## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,(3,3),padding='same',input_shape=x_train.shape[1:]))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64,(3,3),padding='same'))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(128)) #output_dim=100,activation=relu
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))
#輸出
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=10,validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 109s 2ms/step - loss: 1.2104 - accuracy: 0.5788 - val_loss: 1.2672 - val_accuracy: 0.5796
Epoch 2/10
50000/50000 [==============================] - 109s 2ms/step - loss: 0.9156 - accuracy: 0.6824 - val_loss: 0.9982 - val_accuracy: 0.6561
Epoch 3/10
50000/50000 [==============================] - 109s 2ms/step - loss: 0.8044 - accuracy: 0.7226 - val_loss: 0.9045 - val_accuracy: 0.6849
Epoch 4/10
50000/50000 [==============================] - 109s 2ms/step - loss: 0.7274 - accuracy: 0.7451 - val_loss: 0.8659 - val_accuracy: 0.7073
Epoch 5/10
50000/50000 [==============================] - 109s 2ms/step - loss: 0.6698 - accuracy: 0.7681 - val_loss: 1.0024 - val_accuracy: 0.6736
Epoch 6/10
50000/50000 [==============================] - 109s 2ms/step - loss: 0.6219 - accuracy: 0.7842 - val_loss: 0.9550 - val_accuracy: 0.6855
Epoch 7/10
50000/50000 [==============================] - 109s

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[3.40866856e-04, 8.17477558e-05, 6.61951080e-02, 7.99496412e-01,
        2.10467745e-02, 6.26608846e-04, 1.03456005e-01, 1.53934211e-03,
        7.21699512e-03, 1.81122033e-07]], dtype=float32)